### Introduction
- Neowiz 사의 다른 게임인 "킹덤오브히어로 : Tactics War" 의 google playstore review 전체 crawling
- 작동 방식은 브라운더스트 crawling과 동일

#### Import libraries

In [9]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
driver = webdriver.Chrome('./chromedriver')
review_url = 'https://play.google.com/store/apps/details?id=com.neowiz.game.koh&showAllReviews=true'
driver.get(review_url) #접속할 Review 

#### Web page control

In [11]:
time_now = time.time()
flag=0 
while 1:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    try:
        loadMore=driver.find_element_by_xpath("//*[contains(@class,'U26fgb O0WRkf oG5Srb C0oVfc n9lfJ')]").click()
    except:
        time.sleep(1)
        flag=flag+1
        if flag >= 10: 
            break
    else:
        flag=0
print("{} seconds spent to viewing all reviews".format(time.time()-time_now))

344.3110375404358 seconds spent to viewing all reviews


In [18]:
table_dict = {}
date_list = []
contents_list = []
stars_list = []

#reviews=driver.find_elements_by_xpath("//*div[@class='d15Mdf bAhLNe']")
reviews=driver.find_elements_by_xpath("//*[@id='fcxH9b']//div[@class='d15Mdf bAhLNe']")

In [19]:
print(len(reviews))

2231


#### Rating

In [16]:
star_cnt = 0
for review in reviews: #text로 설명된 rating
    soup=BeautifulSoup(review.get_attribute("innerHTML"),"lxml")
    ratings=soup.find('div',role='img').get('aria-label')
    stars_list.append(ratings)
    star_cnt += 1

print("{}개의 review를 수집합니다.".format(star_cnt))

2231개의 review를 수집합니다.


#### Review text

In [6]:
for i in range(1,star_cnt+1): #review text
    review_txt = driver.find_elements_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div/div['+ str(i) + ']/div/div[2]/div[2]/span[1]')[0].text
    contents_list.append(review_txt)
    if i%100==0:
        print("{}th review text".format(i))

100th review text
200th review text
300th review text
400th review text
500th review text
600th review text
700th review text
800th review text
900th review text
1000th review text
1100th review text
1200th review text
1300th review text
1400th review text
1500th review text
1600th review text
1700th review text
1800th review text
1900th review text
2000th review text
2100th review text
2200th review text


#### Review date

In [7]:
for i in range(1,star_cnt+1): #review date
    review_date = driver.find_elements_by_xpath('//*[@id="fcxH9b"]/div[4]/c-wiz/div/div[2]/div/div[1]/div/div/div[1]/div[2]/div/div['+ str(i) + ']/div/div[2]/div[1]/div[1]/div/span[2]')[0].text
    date_list.append(review_date)
    if i%100==0:
        print("{}th review date".format(i))

100th review date
200th review date
300th review date
400th review date
500th review date
600th review date
700th review date
800th review date
900th review date
1000th review date
1100th review date
1200th review date
1300th review date
1400th review date
1500th review date
1600th review date
1700th review date
1800th review date
1900th review date
2000th review date
2100th review date
2200th review date


#### Saving Data Frame 

In [8]:
table_dict['time'] = date_list
table_dict['review_txt'] = contents_list
table_dict['stars_txt'] = stars_list

review_df = pd.DataFrame.from_dict(table_dict)
review_df['time'] = review_df.loc[:,'time'].str.replace(" ","")
review_df['time'] = pd.to_datetime(review_df.loc[:,'time'], format = '%Y년%m월%d일')
print('가장 최근 작성 review 날짜 : {} '.format(review_df['time'].max()))
print('가장 오래전 작성 review 날짜 : {} '.format(review_df['time'].min()))

review_df['ratings'] = review_df['stars_txt'].str[10]
review_df['ratings'] = review_df['ratings'].astype(int)

print('review 별 평점 분포', '\n', review_df['ratings'].value_counts())
print('review 별 평점 분포 %', '\n', review_df['ratings'].value_counts(normalize = True))
    
review_df.to_csv('review_crwl_kingdomhero.csv', index = False)

driver.close()


 
 

가장 최근 작성 review 날짜 : 2019-05-23 00:00:00 
가장 오래전 작성 review 날짜 : 2018-09-05 00:00:00 
review 별 평점 분포 
 5    1364
4     400
1     237
3     152
2      76
Name: ratings, dtype: int64
review 별 평점 분포 % 
 5    0.611934
4    0.179453
1    0.106326
3    0.068192
2    0.034096
Name: ratings, dtype: float64
